In [4]:
import numpy as np
import pickle

## LOAD THE TRAINING DATA

In [5]:
#LOAD THE TRAIN AND TEST
X = np.load('x_train_data.npy')
y = np.load('y_validate_data.npy')

In [6]:
# LOAD DICTIONAIRY

In [7]:
with open('char_to_ix.pickle', 'rb') as handle:
    char_to_ix = pickle.load(handle)
with open('ix_to_char.pickle', 'rb') as handle:
    ix_to_char = pickle.load(handle)


In [8]:
ix_to_char[45]

'म'

In [19]:
SEQ_LENGTH = 120
VOCAB_SIZE = 87

### TRAIN THE DATA (Currently , to see how training works , take epoch to 1)

In [9]:
from keras.models import Sequential, Model,load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import categorical_accuracy
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback

/glob/intel-python/versions/2018u2/intelpython3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
# PREPARE THE MODEL

In [12]:
model = Sequential()

model.add( LSTM(256,return_sequences= True , input_shape = [SEQ_LENGTH,VOCAB_SIZE]) )
model.add(LSTM(256,input_shape = (None,VOCAB_SIZE),return_sequences=True))

model.add(Dropout(0.6))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop",metrics=['accuracy'])

NameError: name 'SEQ_LENGTH' is not defined

In [ ]:
model.summary()

In [ ]:
# NOTE : INPUT TO MODEL IS BATCH of CHARACTERS , where each sequence is of 120
# OUPTUT OF MODEL IS 120 characters, each represented as one hot vectors of 86 zeros and single 1 

In [ ]:
# SET CALLBACK TO PRINT MODEL OUTPUT AT EACH EPOCH
def on_epoch_end(epoch,logs):
    SEQ_LENGTH = 120
    VOCAB_SIZE = 87
    print('\n')
    print("---generating text after epoch using USELESS function: ", epoch)
    print('\n')
    file1 = open('another_function_text.txt','a')
    file1.write('\n')
    file1.write('--------------------sampe {} -----------------'.format(epoch))
    file1.write('\n')
    ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    hold = np.zeros((1, SEQ_LENGTH, VOCAB_SIZE))
    ix = ix[-1]
    for i in range(120):
        print(ix_to_char[ix], end = "")
        hold[0, i, :][ix] = 1
        ix = np.argmax(model.predict(hold[:, :, :])[0][i], 0)
        y_char.append(ix_to_char[ix])
    text = ('').join(y_char)
    file1.write(text)

    file1.write('-------------------END OF {} SAMPLE--------------------'.format(epoch)+'\n')
    print('\n')
   

## TRAIN THE MODEL

In [13]:
# FOR NOW , LET's USE A TRAINED MODEL, as TRAINING TAKES SIGNIFICANT TIME

# epochs = 50
# batch_size = 64

# print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# callbacks=[EarlyStopping(patience=4, monitor='val_loss'),print_callback,
#            ModelCheckpoint(filepath="/home/u8217/lstmShakes/RNN_Text_Generation_Tensorflow-master/news_total_model-{epoch:02d}-{val_acc:.2f}.h5" , 
#            monitor='val_loss', verbose=1, mode='auto',period=4,save_weights_only = False)]

# model.fit(X,y,validation_split=0.1,batch_size= batch_size ,callbacks=callbacks,epochs=epochs)
# model.save('final_model_news_generator.h5')

# model.fit(X,y,batch_size= batch_size,verbose= 1,epochs = 50 )


In [14]:
# LOAD TRAINED MODEL
trainedM = load_model('corrected_news-16-0.59.h5',compile=True)

## GENERATE TEXT USING OUR LANGUAGE GENERATION MODEL

In [17]:
# def generate_txt(model):
    
#     randSeed = X[np.random.randint(0,VOCAB_SIZE)]
#     charlist = [ix_to_char[i] for i in np.argmax(randSeed,axis = 1) ] 
    
#     print('...........SEED TEXT.............')
#     seed_text = "".join(charlist)
#     print(seed_text)
#     print('\n')
   
#     pattern = randSeed.reshape((1,SEQ_LENGTH,VOCAB_SIZE))
#     for i in range(200):
#         prediction = model.predict(pattern)
#         myword = prediction[:,-1,:]
        
#         wordind = np.argmax(myword)
#         print(ix_to_char[wordind], end = "")
#         lastpred  = myword.reshape(1,1,VOCAB_SIZE)
#         charlist.append(ix_to_char[wordind])
#         pattern = np.hstack((pattern[:,1:,:],lastpred))
# #         pattern = pattern[:,i+1:SEQ_LENGTH+i+1,: ]
#     ('').join(charlist)


# CODE FOR GENEARTING NEW TEXT FROM MODEL
def generate_text(model, length= 120,seedStart = 'स',seed = False):
    if seed :
        ix = [char_to_ix[seedStart]]
    else :
        ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    ix = ix[-1]
    for i in range(120):
        print(ix_to_char[ix], end="")
        X[0, i, :][ix] = 1
        ix = np.argmax(model.predict(X[:, :, :])[0][i], 0)
        y_char.append(ix_to_char[ix])
    return ('').join(y_char)

In [20]:
generate_text(trainedM,seedStart='व',seed = True)

वास स्थापना गरेको छ                                                                                                     

'वास स्थापना गरेको छ                                                                                                      '

In [21]:
generate_text(trainedM,seedStart='स',seed = True)

स्थान गर्ने कारण प्रतिक्रिया दिएका छन्                                                                                  

'स्थान गर्ने कारण प्रतिक्रिया दिएका छन्                                                                                   '

Our lstm cells learned to generate new characters completely on their own, given a input starting point.
If you observe carefully , you will see the sentence actually makes sense.
Since we used news article , we get similar vibe from this generated text. 
I trained the model in a smaller dataset for just 24 epochs. Training took 4 hours . So, Basically , You need to train on a lot bigger dataset for model to be more perfect.